<a href="https://colab.research.google.com/github/Rogerio-mack/work/blob/main/Extracao_CNIS_para_revisao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extrator de dados de remunerações - CNIS

Notebook contendo o extrator de remunerações do CNIS

## Importando as libraries

- Utilizamos o pandas para criar o dataframe  
- Utilizamos o pdflib para ler o PDF  
- Utilizamos o re para criar um [Regex](https://pt.wikipedia.org/wiki/Express%C3%A3o_regular)


In [ ]:
import re
import pandas as pd

!pip install pdflib
from pdflib import Document

     |████████████████████████████████| 5.8 MB 23.9 MB/s 


## Extração do PDF

### Extraindo uma página do CNIS 

Para exemplificar estamos utilizando apenas uma página do CNIS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
doc = Document("/content/drive/MyDrive/MackPrevIA/Exemplos de CNIS/CNIS-Cristina Mitiyo Maru Inagaki.pdf")
page1 = doc.get_page(1).lines
print(page1)

['Página 1 de 9', 'INSS - INSTITUTO NACIONAL DO SEGURO SOCIAL', 'CNIS - Cadastro Nacional de Informações Sociais', 'Extrato Previdenciário                                                                         27/04/2021 19:02:47', 'Identificação do Filiado', 'NIT: 122.05604.68-8                              CPF: 088.153.748-92              Nome: CRISTINA MITIYO MARU INAGAKI', 'Data de nascimento: 03/01/1965                                                    Nome da mãe: ALICE AKIY KAWAMATA', 'Relações Previdenciárias', 'Seq.         NIT           Código Emp.        Origem do Vínculo                                   Data Início     Data Fim         Tipo Filiado no Vínculo       Últ. Remun. Indicadores', '1    122.05604.68-8       47.192.091        COTIA TRABALHO TEMPORARIO LTDA EM                   13/05/1985     28/06/1985               Empregado                           AVRC-DEF', 'RECUPERACAO JUDICIAL', 'Seq.         NIT           Código Emp.        Origem do Vínculo             

### Pegando os dados iniciais do cliente

Extraindo a informação inicial do cliente utilizando index da lista de strings e armazenando em um dataframe

In [ ]:
client = {
    "Nome": page1[5].split("Nome: ", 1)[1],
    "NIT": page1[5].split("NIT: ", 1)[1][:14],
    "CPF": page1[5].split("CPF: ", 1)[1][:14],
    "Nome da mãe": page1[6].split("Nome da mãe: ", 1)[1],
    "Data de nascimento": page1[6].split("Data de nascimento: ", 1)[1][:10],
}

client = pd.DataFrame.from_records(client, index=[0])
client["Data de criação do arquivo"] = page1[3].split("Extrato Previdenciário")[1].strip()

client.head()

,CPF,Data de nascimento,NIT,Nome,Nome da mãe,Data de criação do arquivo
0,088.153.748-92,03/01/1965,122.05604.68-8,CRISTINA MITIYO MARU INAGAKI,ALICE AKIY KAWAMATA,27/04/2021 19:02:47


### Declarando funções auxíliares

Essas funções são utilizadas para auxíliar e simplificar as extrações de remunerações.

#### _check_empty_values_ 
 
Verifica os valores vazios nas linhas de relações previdenciárias para manter o padrão da tabela,
caso o valor não seja encontrado adicionamos o valor ```None```. Essa função retorna
uma lista com os dados que irá virar uma tabela no futuro.

E.g

```['1', '108.87248.60-5', '49.282.643/0001-64', 'AMELIA DA CONCEICAO DE OLIVEIRA', '02/07/1979', '01/02/1980', 'Empregado', None, 'AVRC-DEF']```


In [ ]:
def check_empty_values(val: list) -> list:
    for i in range(5, len(val)):

        # Data Fim
        if i == 5 and "/" not in val[i]:
            val.insert(i, None)

        # Últ. Remun.
        elif i == 7:
            if "/" not in val[i]:
                val.insert(i, None)

    # Indicadores e/ou Últ. Remun. se for preciso
    while len(val) < 9:
        val.append(None)

    val.append([])
    return val

#### _remove_blank_spaces_ 

Essa função utiliza o [Regex]() para remover os espaços da string, os subtituindo pelo carácter &

E.g **_essa  string_** vai se tornar em **_essa&string_**  
  
  
Assim conseguimos saber quando uma palavra inicia e termina.



In [ ]:
# Substitui espaços em brancos maiores que 1 pelo caracter &
def remove_blank_spaces(line: str) -> str:
    return re.sub('  +', '&', line)

#### _get_colum_index_

Essa função retorna o index da coluna desejada. 

In [ ]:
# Retorna o index da coluna dada a palavra
def get_colum_index(page: list, var: str) -> int:
    for line in page:
        if var in line:
            return page.index(line)

    return -1

#### _check_empty_values_rem_

Essa função faz a mesma coisa que _check_empty_values_ mas para verificar as Remunerações ao invés de Relações Previdenciárias

Verifica a estrutura e os espaços vázios nas remunerações, retornando uma lista com os dados, isso mantém o mesmo padrão de lista para todas as remunerações.

E.g

```['01/1982', '7.705,01', None, '02/1982', '24.260,95', None, '03/1982', '21.119,00', None]```

In [ ]:
# Verifica os valores vazios
def check_empty_values_rem(val: list) -> list:

    # For loop com entre os valores das remunerações
    for index in range(0, 9, 3):

        if index < len(val):  # Verificando a existência dos dados

            if val[index][2] == '/':  # Verificando a existência da data

                if val[index + 1][-3] == ',':  # Verificando a existência da remuneração

                    if index + 2 < len(val):  # Verificando a existência dos dados

                        if val[index + 2][
                                2] == '/':  # Verificando a existência dos indicadores - LBYL
                            val.insert(index + 2, None)

                    else:  # Inserindo o valor vazio caso os dados sejam inexistentes
                        val.append(None)

        else:
            for i in range(3):
                val.append(None)

    return val  # Retornando a lista

#### _rem_list_to_json_normalized_

Adiciona a coluna de ```Seq.``` para normalizar e conectar as remunerações. Essa função retorna uma lista de dicionários.

E.g

```[{'Competência': '04/1982', 'Remuneração': '19.427,01', 'Indicadores': None,Seq.': '4'},```
  ```{'Competência': '05/1982', 'Remuneração': '15.678,94', 'Indicadores': None, 'Seq.': '4'}]```

In [ ]:
# Função que normaliza o json
def rem_list_to_json_normalized(indexes: list, val: list, seq: str) -> list:
    rem_dict = {}
    rem_list = []

    for i in range(0, 9, 3):
        if val[i]:
            rem_dict = {
                index: value
                for index, value in zip(indexes[:3], val[i:i + 3])
            }
            rem_dict["Seq."] = seq
            rem_list.append(rem_dict)

    return rem_list

### Variáveis adicionais

Aqui fazemos um tratamento básico para pegar as columas para os dataframes

In [ ]:
# Filtrando Relações Previdenciárias e removendo colunas
rel_prev = [line for line in page1 if line[0].isnumeric()]

# Pegando colunas Padrões das Relações
colunas = page1[get_colum_index(page1, "Seq.")]
colunas = remove_blank_spaces(colunas)
# colunas = colunas[:len(colunas) - 12] + "&Indicadores&Remuneracoes"
colunas = colunas[:len(colunas) - 12] + "&Indicadores"  # Versao normalizada
indexes_rel = colunas.split("&")

# Pegando colunas Padrões das Remunerações
colunas = page1[get_colum_index(page1, "Competência")]
colunas = remove_blank_spaces(colunas)
indexes_rem = colunas.split("&")

ult_rel: int = 0
seq: int
rem_final: list = []
df_relacao: list = []
df_remuneracao: list = []

### Extraíndo os dados finais

Aqui temos a extração de dados da remuneração e das relações previdenciárias

#### _clean_up_data_normalized / clean_data_

Nessa função os dados são limpos e transformamos os dicionários em tabelas do Pandas

In [ ]:
# Função para normalizar as remunerações
def clean_up_data_normalized(dados: list, linha: list) -> None:
    global seq, rem_final

    line = remove_blank_spaces(linha).split("&")  # Pegando a linha

    if line[0].isnumeric(
    ):  # Verificando se o primeiro valor da linha é numérico
        values: list = check_empty_values(line)
        seq = line[0]

        result: dict = {index: value for index, value in zip(indexes_rel, values)}
        df_relacao.append(pd.DataFrame.from_records(result, index=[0]))

    else:
        rem: list = check_empty_values_rem(line)
        rem_json: list = rem_list_to_json_normalized(indexes_rem, rem, seq)

        dfs: list = [pd.DataFrame.from_records(elem, index=[0]) for elem in rem_json]
        
        df_remuneracao.append(pd.concat(dfs, ignore_index=True))

# Percorrendo lista de relações
for n in range(len(rel_prev)):
    clean_up_data_normalized(client, rel_prev[n])

# Alterando as opções do print do Dataframe
pd.set_option("display.max_rows", None, "display.max_columns", None)

# print(df_relacao)
df_relacao = pd.concat(df_relacao, ignore_index=True)
df_relacao.drop_duplicates()
print(df_relacao.head())
print("\n\n\n\n")
df_remuneracao = pd.concat(df_remuneracao, ignore_index=True)
print(df_remuneracao)


          Código Emp.    Data Fim          Data Início Indicadores  \
0          47.192.091  28/06/1985           13/05/1985    AVRC-DEF   
1  56.991.888/0008-00  12/07/1991           08/09/1987     ACNISVR   
2  61.192.571/0002-40  15/07/1991   JOHNSON COMERCIO E   Empregado   

              NIT                      Origem do Vínculo Seq.  \
0  122.05604.68-8      COTIA TRABALHO TEMPORARIO LTDA EM    1   
1  122.05604.68-8  ENGESA ENGENHEIROS ESPECIALIZADOS S A    2   
2  122.05604.68-8                               JOHNSON     3   

  Tipo Filiado no Vínculo Últ. Remun.  
0               Empregado        None  
1               Empregado     12/1989  
2              07/05/2012        None  





   Competência Indicadores Remuneração Seq.
0      09/1987        None   15.051,99    2
1      10/1987        None   20.555,00    2
2      11/1987        None   26.979,99    2
3      12/1987        None   28.374,99    2
4      01/1988        None   30.982,98    2
5      02/1988        None   

#### _get_rem_

Essa função será utilizada para pegar as remunerações da página e utiliza a função auxíliar _clean_data_.


In [ ]:
def clean_data(seq, linha, df_remuneracao):

  line = remove_blank_spaces(linha).split("&")

  if line[0].isnumeric():
      seq = line[0]

  else:

    rem: list = check_empty_values_rem(line)
    rem_json: list = rem_list_to_json_normalized(indexes_rem, rem, seq)

    dfs: list = [pd.DataFrame.from_records(elem, index=[0]) for elem in rem_json]
    
    df_remuneracao.append(pd.concat(dfs, ignore_index=True))  

  return seq



def get_rem(page: list) -> pd.DataFrame:

    rel_prev = [line for line in page if line[0].isnumeric()]

    colunas = page[get_colum_index(page, "Competência")]
    colunas = remove_blank_spaces(colunas)
    indexes_rem = colunas.split("&")
    df_remuneracao: list = []
    seq = 0

    for n in range(len(rel_prev)):
      seq = clean_data(seq, rel_prev[n], df_remuneracao)

    df_remuneracao = pd.concat(df_remuneracao, ignore_index=True)

    return df_remuneracao

In [ ]:
df_remuneracao = get_rem(page1)


print(df_remuneracao.head(10))

  Competência Indicadores Remuneração Seq.
0     09/1987        None   15.051,99    2
1     10/1987        None   20.555,00    2
2     11/1987        None   26.979,99    2
3     12/1987        None   28.374,99    2
4     01/1988        None   30.982,98    2
5     02/1988        None   38.905,99    2
6     03/1988        None   45.205,00    2
7     04/1988        None   67.807,99    2
8     05/1988        None   78.786,98    2
9     06/1988        None   92.716,99    2


# Dados do Beneficiário
---

### FAZER

None

### PENDENTE

None

In [ ]:
client 

,CPF,Data de nascimento,NIT,Nome,Nome da mãe,Data de criação do arquivo
0,088.153.748-92,03/01/1965,122.05604.68-8,CRISTINA MITIYO MARU INAGAKI,ALICE AKIY KAWAMATA,27/04/2021 19:02:47


# Dados dos Vínculos
---

### FAZER
1. Programa trabalha uma única página: implementar ´for´ para trabalhar todas as páginas do documento. case: /content/drive/MyDrive/MackPrevIA/Exemplos de CNIS/CNIS Rogerio extrato.pdf 

2. Erro na tratativa de nomes de Vínculo com &. case: /content/drive/MyDrive/MackPrevIA/Exemplos de CNIS/CNIS-Cristina Mitiyo Maru Inagaki.pdf 

>> Responsável: Lucas e Daniel

### PENDENTE
1. Para Origens do vínculo com mais de uma linha trunca para uma única linha. case: /content/drive/MyDrive/MackPrevIA/Exemplos de CNIS/CNIS - Claudia.pdf 

2. Benefício Auxilio Acidente não tratado. case: /content/drive/MyDrive/MackPrevIA/Exemplos de CNIS/CNIS - Jayr Antonio da Silva.pdf 

In [ ]:
df_relacao

,Código Emp.,Data Fim,Data Início,Indicadores,NIT,Origem do Vínculo,Seq.,Tipo Filiado no Vínculo,Últ. Remun.
0,47.192.091,28/06/1985,13/05/1985,AVRC-DEF,122.05604.68-8,COTIA TRABALHO TEMPORARIO LTDA EM,1,Empregado,None
1,56.991.888/0008-00,12/07/1991,08/09/1987,ACNISVR,122.05604.68-8,ENGESA ENGENHEIROS ESPECIALIZADOS S A,2,Empregado,12/1989
2,61.192.571/0002-40,15/07/1991,JOHNSON COMERCIO E,Empregado,122.05604.68-8,JOHNSON,3,07/05/2012,None


# Dados das Remunerações
---

### FAZER
1. Programa trabalha uma única página: implementar ´for´ para trabalhar todas as páginas do documento. case: /content/drive/MyDrive/MackPrevIA/Exemplos de CNIS/CNIS Rogerio extrato.pdf 

### PENDENTE

1. Presença de Vínculos sem Remuneração. case: /content/drive/MyDrive/MackPrevIA/Exemplos de CNIS/CNIS - Jayr Antonio da Silva.pdf 

In [ ]:
df_remuneracao

,Competência,Indicadores,Remuneração,Seq.
0,09/1987,None,"15.051,99",2
1,10/1987,None,"20.555,00",2
2,11/1987,None,"26.979,99",2
3,12/1987,None,"28.374,99",2
4,01/1988,None,"30.982,98",2
5,02/1988,None,"38.905,99",2
6,03/1988,None,"45.205,00",2
7,04/1988,None,"67.807,99",2
8,05/1988,None,"78.786,98",2
9,06/1988,None,"92.716,99",2
